# Your goal is to predict the values of single unit properties using the observations from 2017.

## From the zillow database for all 'Single Family Residential' properties\
### Acquire:
bedroomcnt\
bathroomcnt\
calculatedfinishedsquarefeet\
taxvaluedollarcnt\
yearbuilt\
taxamount\
fips

In [1]:
import env 
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os


In [2]:
# url = env.get_db_url('zillow')

# df_zillow = pd.read_sql(('''select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt,
#                          taxamount, fips
#                          from properties_2017
#                          join propertylandusetype
#                              using (propertylandusetypeid)
#                          WHERE propertylandusedesc = ("Single Family Residential")'''), url)

In [5]:
def get_zillow():
    '''
    This function acquires zillow data from Codeup MySQL
    '''
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else:
        url = env.get_db_url('zillow')
        
        df_zillow = pd.read_sql(('''select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt,
                                 taxamount, fips
                                 from properties_2017
                                 left join propertylandusetype
                                     using (propertylandusetypeid)
                                 WHERE propertylandusedesc = ("Single Family Residential")'''), url)
        df_zillow.to_csv(filename)
    return df_zillow

In [6]:
df_zillow=get_zillow()

In [7]:
# initial look at data
df_zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


## Walk through the summarization and cleaning steps

In [8]:
# Look at data with info to get dtypes
df_zillow.info()

# Title changes: change bedroomcnt to bedrooms, bathroomcnt to bathrooms, calculatedfinishedsquarefeet to sqft, 
# taxvaluedollrcnt to appraisal, yearbuilt to year_built, taxamount to taxes, and fips to city_code

# Dtype changes: change yearbuilt & bedroomcnt.  Fips to obj dtype.  

<class 'pandas.core.frame.DataFrame'>
Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 131.4 MB


In [9]:
# Rename columns
column_name_changes = {'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 
                       'sqft', 'taxvaluedollarcnt': 'appraisal', 'yearbuilt': 'year built', 'taxamount': 'taxes',
                      'fips': 'county'}
df_zillow.rename(columns=column_name_changes, inplace=True)

In [10]:
# get idea for the size of the dataset
df_zillow.shape

(2152863, 7)

In [11]:
# look at summary stats
df_zillow.describe().T

,count,mean,std,min,25%,50%,75%,max
bedrooms,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathrooms,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
sqft,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
appraisal,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
year built,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxes,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
county,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [12]:
df_zillow.dtypes

bedrooms      float64
bathrooms     float64
sqft          float64
appraisal     float64
year built    float64
taxes         float64
county        float64
dtype: object

In [13]:
# look at nulls
df_zillow.isnull().sum()

bedrooms        11
bathrooms       11
sqft          8484
appraisal      493
year built    9337
taxes         4442
county           0
dtype: int64

## Examine each feature

In [14]:
df_zillow[df_zillow.bedrooms.isnull()]
# going to convert NaNs to 0 and drop

,bedrooms,bathrooms,sqft,appraisal,year built,taxes,county
107763,NaN,NaN,NaN,67366.0,1926.0,780.54,6059.0
118612,NaN,NaN,NaN,43992.0,1946.0,541.64,6059.0
193993,NaN,NaN,1348.0,840698.0,1952.0,NaN,6059.0
1141339,NaN,NaN,200.0,188972.0,NaN,NaN,6037.0
1324608,NaN,NaN,990.0,435000.0,1906.0,NaN,6037.0
1442975,NaN,NaN,NaN,273196.0,NaN,NaN,6037.0
1647346,NaN,NaN,400.0,28347.0,1954.0,NaN,6037.0
1701026,NaN,NaN,NaN,407930.0,1926.0,NaN,6037.0
1722707,NaN,NaN,NaN,477161.0,NaN,NaN,6037.0
1776422,NaN,NaN,NaN,38855.0,NaN,NaN,6037.0


In [15]:
df_zillow = df_zillow.dropna()

In [16]:
# dtype changes
df_zillow['year built'] = df_zillow['year built'].astype(int)
df_zillow.bedrooms = df_zillow.bedrooms.astype(int)
df_zillow.county = df_zillow.county.astype(object)

In [17]:
df_zillow[df_zillow.bedrooms == 0]

,bedrooms,bathrooms,sqft,appraisal,year built,taxes,county
11,0,0.0,1200.0,5328.0,1972,91.60,6037.0
14,0,0.0,171.0,6920.0,1973,255.17,6037.0
15,0,0.0,203.0,14166.0,1960,163.79,6037.0
866,0,0.0,220.0,13840.0,1988,327.80,6037.0
2306,0,0.0,1776.0,79574.0,1989,1137.90,6037.0
...,...,...,...,...,...,...,...
2146660,0,0.0,676.0,234580.0,1922,3013.57,6037.0
2149054,0,0.0,938.0,56733.0,1978,986.86,6037.0
2152505,0,0.0,240.0,31234.0,1942,414.04,6037.0
2152704,0,1.0,1490.0,152000.0,1930,3614.40,6037.0


In [18]:
df_zillow.bedrooms.value_counts(dropna=False)
# no bedrooms...studios?  would have bathroom still.  check bathrooms

bedrooms
3     962944
4     633608
2     334221
5     150671
6      25117
1      22895
7       4792
0       4397
8       1103
9        290
10       118
11        34
13        15
12        12
14         7
15         5
18         3
16         2
25         1
Name: count, dtype: int64

In [19]:
df_zillow.bathrooms.value_counts(dropna=False)

bathrooms
2.00     942463
3.00     422398
1.00     412582
2.50     142827
4.00      82039
1.50      31157
3.50      28464
5.00      28306
4.50      19474
6.00      10717
5.50       6201
7.00       4381
0.00       4274
8.00       1681
6.50       1330
9.00        707
7.50        382
10.00       322
11.00       145
8.50        108
12.00        73
9.50         50
13.00        39
14.00        25
15.00        17
0.50         16
10.50        14
16.00        12
18.00         8
20.00         6
17.00         4
1.75          3
12.50         3
11.50         3
19.50         1
14.50         1
32.00         1
19.00         1
Name: count, dtype: int64

In [20]:
df_zillow.isnull().sum()

bedrooms      0
bathrooms     0
sqft          0
appraisal     0
year built    0
taxes         0
county        0
dtype: int64

In [21]:
df_zillow.bedrooms.value_counts(dropna=False)

bedrooms
3     962944
4     633608
2     334221
5     150671
6      25117
1      22895
7       4792
0       4397
8       1103
9        290
10       118
11        34
13        15
12        12
14         7
15         5
18         3
16         2
25         1
Name: count, dtype: int64

In [22]:
df_zillow = df_zillow[(df_zillow['bedrooms'] != 0) & (df_zillow['bathrooms'] != 0)]

In [23]:
df_zillow.bedrooms.value_counts(dropna=False)

bedrooms
3     962638
4     633362
2     334078
5     150598
6      25104
1      22752
7       4791
8       1102
9        290
10       118
11        33
13        15
12        12
14         6
15         5
16         2
18         2
25         1
Name: count, dtype: int64

In [24]:
df_zillow[df_zillow.bedrooms == 0]

,bedrooms,bathrooms,sqft,appraisal,year built,taxes,county


In [25]:
df_zillow['bedrooms'].unique()

array([ 4,  3,  5,  2,  1,  6,  7,  8,  9, 25, 10, 11, 14, 15, 13, 12, 16,
       18])

In [26]:
def prep_zillow(df):
    '''
    this function prepares zillow data by changing column names, a few dtypes, dropping nulls, and houses with 0 bedrooms and 0 bathrooms.
    '''
    column_name_changes = {'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 
                       'sqft', 'taxvaluedollarcnt': 'appraisal', 'yearbuilt': 'year built', 'taxamount': 'taxes',
                      'fips': 'county'}
    df.rename(columns=column_name_changes, inplace=True)
    
    df = df.dropna()
    
    # dtype changes
    df['year built'] = df['year built'].astype(int)
    df.bedrooms = df.bedrooms.astype(int)
    df.county = df.county.astype(object)
    
    df = df[(df['bedrooms'] != 0) & (df['bathrooms'] != 0)]
    
    return df


In [27]:
df_zillow=prep_zillow(df_zillow)
df_zillow.head()

,bedrooms,bathrooms,sqft,appraisal,year built,taxes,county
4,4,2.0,3633.0,296425.0,2005,6941.39,6037.0
6,3,4.0,1620.0,847770.0,2011,10244.94,6037.0
7,3,2.0,2077.0,646760.0,1926,7924.68,6037.0
18,3,1.0,1244.0,169471.0,1950,2532.88,6037.0
19,3,2.0,1300.0,233266.0,1950,3110.99,6037.0


In [28]:
def split_data(df):
    '''
   
    '''
    train, validate_test = train_test_split(df,
                                            train_size=0.60,
                                            random_state=123,
                                            )

 
    validate, test = train_test_split(validate_test,
                                      test_size=0.50,
                                      random_state=123,
                                      )

    return train, validate, test

In [29]:
train, validate, test=split_data(df_zillow)
train.shape, validate.shape, test.shape


((1280945, 7), (426982, 7), (426982, 7))

## Create final function called wrangle_zillow

In [43]:
def wrangle_function():
    train, validate, test=split_data(prep_zillow(get_zillow()))
    
    return train, validate, test

In [44]:
train, validate, test=wrangle_function()

In [46]:
train.shape

(1280945, 7)